In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.subplots as sp
from influxdb_client import InfluxDBClient, Point
from influxdb_client.client.write_api import SYNCHRONOUS
import json
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go

from astral import LocationInfo
from astral.sun import sun
import datetime
import pytz

from scipy.signal import savgol_filter
import plotly.graph_objects as go

import scipy.stats as stats

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dropout, Dense, Lambda
from tensorflow.keras import layers
from tensorflow.keras.models import Model

In [ ]:
#read in the households file with lats and longs + only the ids that are in the database
#json files
file_path = r"C:\Users\samr0\OneDrive - KU Leuven\Documents\!School\master\Thesis\data\households_in_database.json"
#read JSON into a dataFrame
df_households = pd.read_json(file_path)

df_households.head()

# Weather data

In [ ]:
#part 1
file_path = r"C:\Users\samr0\OneDrive - KU Leuven\Documents\!School\master\Thesis\data\aws_10min.csv"

df = pd.read_csv(file_path, index_col='timestamp', parse_dates=True)
cutoff_timestamp = "2022-06-19 04:20:00"

df = df.loc[:cutoff_timestamp]

#part 2
file_path = r"C:\Users\samr0\OneDrive - KU Leuven\Documents\!School\master\Thesis\data\aws_10min_rest.csv"

df2 = pd.read_csv(file_path, index_col='timestamp', parse_dates=True)
cutoff_timestamp = "2022-06-19 04:30:00"

df2 = df2.loc[cutoff_timestamp:]

#combine them
df_combined = pd.concat([df, df2])
df_combined

In [ ]:
#haversine formula to compute the great-circle distance between two points
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # earth radius in km
    lat1 = np.radians(lat1)
    lon1 = np.radians(lon1)
    lat2 = np.radians(lat2)
    lon2 = np.radians(lon2)

    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2.0) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c  #distance in km

In [ ]:
#unique meteostations
df_unique_stations = df_combined.drop_duplicates(subset="code", keep="first")
df_unique_stations[['lat', 'lon']] = df_unique_stations['the_geom'].str.extract(r'POINT \(([^ ]+) ([^ ]+)\)').astype(float)

df_unique_stations

In [ ]:
pd.set_option('display.max_columns', None)

household_id = "7847f5f7"
row = df_households[df_households["id"] == household_id]
lat = row["latitude"].values[0]
lon = row["longitude"].values[0]

df_unique_stations['distance_km'] = df_unique_stations.apply(lambda row: haversine(lat, lon, row['lat'], row['lon']), axis=1)
df_unique_stations = df_unique_stations.sort_values("distance_km")
df_unique_stations.index.names = ['_time']
df_unique_stations

In [ ]:
#pin figure import for legend map
import base64

image_path = "C:/Users/samr0/OneDrive - KU Leuven/Documents/!School/master/Thesis/notebooks/graphics-large-blue-pin-png.png"

with open(image_path, 'rb') as f:
    img_base64 = base64.b64encode(f.read()).decode('utf-8')

#create an <img> tag with the base64 image
img_html = f'<img src="data:image/png;base64,{img_base64}" style="height:16px; vertical-align:middle; margin-right:8px;">'

In [ ]:
import folium

#set the center of the map to the average location or a specific point
map_center = [df_unique_stations['lat'].mean(), df_unique_stations['lon'].mean()]
map = folium.Map(location=map_center, zoom_start=8)

df_top3 = df_unique_stations.head(3)

#add markers for each unique station
for index, row in df_unique_stations.iterrows():
    popup_text = f"Station Code: {row['code']}<br>Distance: {round(row['distance_km'], 2)} km"
    folium.Marker([row['lat'], row['lon']], popup=popup_text).add_to(map)
    
folium.CircleMarker(
    location=[lat, lon],
    radius=6,
    color='red',
    fill=True,
    fill_color='red',
    fill_opacity=0.9,
    popup="Target Location"
).add_to(map)

for index, row in df_top3.iterrows():
    folium.PolyLine(
        locations=[[row['lat'], row['lon']], [lat, lon]],
        color='blue',
        weight=2,
        opacity=0.7,
        dash_array='5, 5'  # Creates dashed effect: 5px line, 5px gap
    ).add_to(map)

legend_html = f"""
<div style="
    position: fixed; 
    bottom: 200px; left: 50px; width: 200px; height: 120px; 
    background-color: white; 
    border:2px solid grey; 
    z-index:9999;
    font-size:14px;
    padding: 10px;
    box-shadow: 2px 2px 5px rgba(0,0,0,0.3);
">
    <b>Legend</b><br>
    <div style="margin-top: 5px;">
        <span style="
            display:inline-block;
            width: 12px;
            height: 12px;
            background-color: red;
            border-radius: 50%;
            margin-right: 8px;
            vertical-align: middle;
        "></span>
        Target Location
    </div>
    <div>
        <span style="
            display:inline-block;
            width: 12px;
            height: 2px;
            background-color: blue;
            margin: 7px 8px 0 0;
            vertical-align: middle;
        "></span>
        Distance Line
    </div>
    <div>
        {img_html}
        Station Marker
    </div>
</div>
"""

map.get_root().html.add_child(folium.Element(legend_html))


#display the map
map

In [ ]:
# add the ghi values of the database
file_path = r"C:\Users\samr0\OneDrive - KU Leuven\Documents\!School\master\Thesis\data\meteoStationsDatabaseData.csv"

meteoStationsData = pd.read_csv(file_path, index_col='_time', parse_dates=True)
meteoStationsData["code"] = meteoStationsData["nodeId"].str[-4:].astype(int)
meteoStationsData

In [ ]:
codesInDatabase = [6434, 6438, 6455, 6459, 6464, 6472, 6477, 6484]
#get only data of stations in the database
df_meteo = df_combined[df_combined["code"].isin(codesInDatabase)]
df_meteo

In [ ]:
#join the ghi data with the rest of the weather data

df_meteo.index = pd.to_datetime(df_meteo.index)
#reset index to merge on both timestamp and code
meteoStationsData_reset = meteoStationsData.reset_index()
df_meteo_reset = df_meteo.reset_index()

df_meteo_reset.rename(columns={'timestamp': '_time'}, inplace=True)

df_meteo_reset['_time'] = pd.to_datetime(df_meteo_reset['_time'])
df_meteo_reset['_time'] = df_meteo_reset['_time'].dt.tz_localize('UTC')


#merge on both timestamp and code
merged_df = pd.merge(meteoStationsData_reset, df_meteo_reset, on=['_time', 'code'], how='inner')

#set timestamp back as index
merged_df.set_index('_time', inplace=True)

merged_df

In [ ]:
#get from the stations in ghi data the unique once and the distance to target node

df_unique_stations = merged_df.drop_duplicates(subset="code", keep="first")
df_unique_stations[['lat', 'lon']] = df_unique_stations['the_geom'].str.extract(r'POINT \(([^ ]+) ([^ ]+)\)').astype(float)
df_unique_stations['distance_km'] = df_unique_stations.apply(lambda row: haversine(lat, lon, row['lat'], row['lon']), axis=1)
df_unique_stations = df_unique_stations.sort_values("distance_km")
df_unique_stations

# if not including the ghi data
merged_df = df_combined
merged_df = merged_df.rename_axis('_time')
merged_df['_time'] = pd.to_datetime(merged_df.index)
merged_df['_time'] = merged_df['_time'].dt.tz_localize('UTC')
merged_df

In [ ]:
#get 3 closest ones and selection
df_closest = df_unique_stations[:3]

unique_codes = df_closest["code"].unique()
unique_codes_list = unique_codes.tolist()

#get all data from nearby stations
df_meteo = merged_df[merged_df["code"].isin(unique_codes_list)]

df_meteo = df_meteo.reset_index()

df_meteo['wind_speed'] = df_meteo['wind_speed_10m'].combine_first(df_meteo['wind_speed_avg_30m'])

df_meteo = df_meteo.drop(columns = ["FID", "the_geom", "temp_grass_pt100_avg", "temp_soil_avg_5cm",
                                            "temp_soil_avg_10cm", "temp_soil_avg_20cm", "temp_soil_avg_50cm",
                                            "qc_flags", "wind_speed_10m", "wind_speed_avg_30m"])

#add lat, lon and distance_km
df_selection = df_closest[['code', 'lat', 'lon', 'distance_km']]


df_meteo = pd.merge(df_meteo, df_selection, on="code", how="left")

df_meteo = df_meteo.set_index("_time")
df_meteo

In [ ]:
#get average weather taking distance into account

#define a function to calculate the weighted average for a given column
def weighted_average(group, weight_column='distance_km'):
    #calculate the weights as the inverse of distance (closer stations get higher weight)
    weights = 1 / group[weight_column]
    
    #compute the weighted average for each column in the group
    return (group.drop(columns=[weight_column]).multiply(weights, axis=0)).sum() / weights.sum()

#drop the nodeId, not a number
df_meteo = df_meteo.drop(columns = ["nodeId"])

#group by timestamp and apply the weighted average function to each group
df_meteo_avg_weighted = df_meteo.groupby(df_meteo.index).apply(weighted_average)

df_meteo_avg_weighted

In [ ]:
df_weather_resampled = df_meteo_avg_weighted.resample('15min').mean()
df_weather_resampled

In [ ]:
#display rows with NaN values
nan_rows = df_weather_resampled[df_weather_resampled.isna().any(axis=1)]

#show the rows containing NaN values
print(nan_rows.sum())

#these are values for which there is no data in the database
df_weather_resampled = df_weather_resampled.dropna()
df_weather_resampled.isna().sum()

# Add the data from the database

In [ ]:
file_path = r"C:\Users\samr0\OneDrive - KU Leuven\Documents\!School\master\Thesis\data\inverter_power_data_7847f5f7_normalised_15min.csv"

df_data = pd.read_csv(file_path, index_col='_time', parse_dates=True)
#df_data.index = df_data.index.tz_convert('Europe/Brussels')

df_data

In [ ]:
#combine data with weather
df_data = df_data.reset_index()

df_data['_time'] = pd.to_datetime(df_data['_time'])
df_data['_time'] = df_data['_time'].dt.tz_localize('UTC')
df_data.set_index('_time', inplace=True)


df_data = pd.merge(df_data, df_weather_resampled, how='left', left_index=True, right_index=True)
df_data

In [ ]:
#extra features

df_data['hour'] = df_data.index.hour
df_data['day_of_week'] = df_data.index.dayofweek
df_data['month'] = df_data.index.month

df_data['hour_sin'] = np.sin(2 * np.pi * df_data['hour'] / 24)
df_data['hour_cos'] = np.cos(2 * np.pi * df_data['hour'] / 24)
df_data['day_of_year'] = df_data.index.dayofyear
df_data['day_of_year_sin'] = np.sin(2 * np.pi * df_data['day_of_year'] / 365)
df_data['day_of_year_cos'] = np.cos(2 * np.pi * df_data['day_of_year'] / 365)

df_data['minute'] = df_data.index.minute

# Encode the 15-minute intervals within an hour
df_data['minute_sin'] = np.sin(2 * np.pi * df_data['minute'] / 60)
df_data['minute_cos'] = np.cos(2 * np.pi * df_data['minute'] / 60)

In [ ]:
df_data

In [ ]:
#remove duplicate indices and only keep first one: for the hour change in Belgium
df_data = df_data.loc[~df_data.index.duplicated(keep='first')]

df_data

In [ ]:
#normalization of other features
#initialize the scaler
scaler = MinMaxScaler()

columns_to_normalize = [
    'precip_quantity', 
    'temp_dry_shelter_avg', 
    'temp_soil_avg',
    'wind_direction',
    'wind_gusts_speed', 
    'humidity_rel_shelter_avg', 
    'pressure', 
    'sun_duration', 
    'short_wave_from_sky_avg', 
    'sun_int_avg', 
    'wind_speed',
'diffuseIrradiance_Wpm2',
'directNormalIrradiance_Wpm2',
'globalHorizontalIrradiance_Wpm2'
]

#apply MinMax scaling only to other features
df_data[columns_to_normalize] = scaler.fit_transform(df_data[columns_to_normalize])
df_data

In [ ]:
#extra shift of 24 hours ago
df_data["normalized_value_shift_24"] = df_data[['normalized_value']].shift(freq='D')
df_data = df_data.dropna()
df_data

# for test including future weather (as approximation of future forecast)

In [ ]:
df_data.loc[:, "precip_quantity_future"] = df_data[['precip_quantity']].shift(freq='-1D')
df_data.loc[:, "temp_soil_avg_future"] = df_data[['temp_soil_avg']].shift(freq='-1D')
df_data.loc[:, "wind_direction_future"] = df_data[['wind_direction']].shift(freq='-1D')
df_data.loc[:, "wind_gusts_speed_future"] = df_data[['wind_gusts_speed']].shift(freq='-1D')
df_data.loc[:, "humidity_rel_shelter_avg_future"] = df_data[['humidity_rel_shelter_avg']].shift(freq='-1D')
df_data.loc[:, "pressure_future"] = df_data[['pressure']].shift(freq='-1D')
df_data.loc[:, "sun_duration_future"] = df_data[['sun_duration']].shift(freq='-1D')
df_data.loc[:, "short_wave_from_sky_avg_future"] = df_data[['short_wave_from_sky_avg']].shift(freq='-1D')
df_data.loc[:, "sun_int_avg_future"] = df_data[['sun_int_avg']].shift(freq='-1D')
df_data.loc[:, "wind_speed_future"] = df_data[['wind_speed']].shift(freq='-1D')
df_data = df_data.dropna()
df_data

# train - cv - test split

In [ ]:
#parameters
input_steps = 4*8
output_steps = 1    #predict one timestep ahead
step_size_train = 1
step_size_test = 1      #shift for testing
step_size_val = 1

#select relevant columns for features and target
features = [
    "normalized_value",
     "day_of_week", "month", 
    "hour_sin", "hour_cos", 
    "day_of_year_sin", "day_of_year_cos",
    "minute_sin", "minute_cos",
    "temp_dry_shelter_avg",
    "normalized_value_shift_24",
    'diffuseIrradiance_Wpm2',
    'directNormalIrradiance_Wpm2',
    'globalHorizontalIrradiance_Wpm2',
    
    #'precip_quantity', 
    #'temp_soil_avg',
    #'wind_direction',
    #'wind_gusts_speed', 
    #'humidity_rel_shelter_avg', 
    #'pressure', 
    #'sun_duration', 
    #'short_wave_from_sky_avg', 
    #'sun_int_avg', 
    #'wind_speed',
    
    #'precip_quantity_future', 
    #'temp_soil_avg_future',
    #'wind_direction_future',
    #'wind_gusts_speed_future', 
    #'humidity_rel_shelter_avg_future', 
    #'pressure_future', 
    #'sun_duration_future', 
    #'short_wave_from_sky_avg_future', 
    #'sun_int_avg_future', 
    #'wind_speed_future',
]

target = "normalized_value"

In [ ]:
def generate_sliding_window(data, input_steps=1440, output_steps=1, feature_columns=None, target_column=None, step_size=1):
    """
    Generate sliding windows for a given range of data.
    - input_steps: Number of timesteps in the input window.
    - output_steps: Number of timesteps to predict.
    - feature_columns: List of feature column names.
    - target_column: Name of the target column.
    - step_size: Shift between consecutive windows.
    """
    X, y, X_indices, y_indices = [], [], [], []
    for i in range(0, len(data) - input_steps - output_steps + 1, step_size):
        # input: feature columns over the input window
        X_window = data[feature_columns].iloc[i:i+input_steps]
        X.append(X_window)
        X_indices.append(data.index[i:i+input_steps])  #store corresponding indices
        
        # target: target column for the output window
        y_window = data[target_column].iloc[i+input_steps:i+input_steps+output_steps]
        y.append(y_window)
        y_indices.append(data.index[i+input_steps:i+input_steps+output_steps])  #store corresponding indices
    
    print("Generated sliding windows - X.size:", len(X), "y.size:", len(y))
    return X, y, X_indices, y_indices

In [ ]:
#define train test sizes
train_size = 20 * 24 * 4  # 20 days in 15 minutes
val_size = 5 * 24 * 4    # 5 days in 15 minutes
test_size = 5 * 24 * 4    # 5 days in 15 minutes

# generate train-test splits dynamically, jump by test_size forward between splits
splits = []
for i in range(0, len(df_data) - train_size - test_size + 1, train_size + val_size + test_size):#if all splits are used for training, there can't be
    #overlap, so jump train_size + test_size
    train_data = df_data.iloc[i:i+train_size]
    val_data = df_data.iloc[i+train_size:i+train_size+val_size]
    test_data = df_data.iloc[i+train_size+val_size:i+train_size+val_size+test_size]
    splits.append((train_data, val_data ,test_data))
print("number of splits: ", len(splits))

In [ ]:
#data split

#storage for training, validation, and test data
all_train_X, all_train_X_indices, all_train_y, all_train_y_indices = [], [], [], []
all_val_X, all_val_X_indices, all_val_y, all_val_y_indices = [], [], [], []
all_test_X, all_test_X_indices, all_test_y, all_test_y_indices = [], [], [], []

# loop over all folds to collect training data
fold = 0
for fold, (train_data, val_data, test_data) in enumerate(splits):
    print(f"Processing Fold {fold + 1}")

    #generate sliding windows for training
    df_train_X, df_train_y, df_train_X_indices, df_train_y_indices = generate_sliding_window(train_data, input_steps, output_steps, features, target, step_size_train)
    
    #generate sliding windows for validation
    df_val_X, df_val_y, df_val_X_indices, df_val_y_indices = generate_sliding_window(val_data, input_steps, output_steps, features, target, step_size_val)
    
    #generate sliding windows for testing
    df_test_X, df_test_y, df_test_X_indices, df_test_y_indices = generate_sliding_window(test_data, input_steps, output_steps, features, target, step_size_test)

    #append training data
    all_train_X.extend(df[features].values for df in df_train_X)
    all_train_X_indices.extend(df.values for df in df_train_X_indices)
    all_train_y.extend(df.values for df in df_train_y)
    all_train_y_indices.extend(df.values for df in df_train_y_indices)

    #append validation data
    all_val_X.extend(df[features].values for df in df_val_X)
    all_val_X_indices.extend(df.values for df in df_val_X_indices)
    all_val_y.extend(df.values for df in df_val_y)
    all_val_y_indices.extend(df.values for df in df_val_y_indices)

    #append test data
    all_test_X.extend(df[features].values for df in df_test_X)
    all_test_X_indices.extend(df.values for df in df_test_X_indices)
    all_test_y.extend(df.values for df in df_test_y)
    all_test_y_indices.extend(df.values for df in df_test_y_indices)

#convert lists to numpy arrays
all_train_X, all_train_y = np.array(all_train_X), np.array(all_train_y)
all_train_X_indices, all_train_y_indices = np.array(all_train_X_indices), np.array(all_train_y_indices)

all_val_X, all_val_y = np.array(all_val_X), np.array(all_val_y)
all_val_X_indices, all_val_y_indices = np.array(all_val_X_indices), np.array(all_val_y_indices)

all_test_X, all_test_y = np.array(all_test_X), np.array(all_test_y)
all_test_X_indices, all_test_y_indices = np.array(all_test_X_indices), np.array(all_test_y_indices)

In [ ]:
all_train_X = all_train_X.astype(np.float32)
all_train_y = all_train_y.astype(np.float32)

all_val_X = all_val_X.astype(np.float32)
all_val_y = all_val_y.astype(np.float32)

all_test_X = all_test_X.astype(np.float32)
all_test_y = all_test_y.astype(np.float32)

In [ ]:
print(np.isnan(all_train_X).sum())  # Count NaNs
print(np.isinf(all_train_X).sum())  # Count Infs

In [ ]:
#initialize a dictionary to store benchmark results
benchmark_results = {}

#function to compute and store evaluation metrics
def evaluate_benchmark(name, y_true, y_pred):
    """Computes MAE, RMSE, MAPE, and R² and stores in a dictionary."""
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    
    #avoid zero division in MAPE
    valid_mask = y_true != 0  
    mape = np.mean(np.abs((y_true[valid_mask] - y_pred[valid_mask]) / y_true[valid_mask])) * 100  
    r2 = r2_score(y_true, y_pred)
    
    #store results in dictionary
    benchmark_results[name] = {
        'MAE': mae,
        'RMSE': rmse,
        'MAPE': mape,
        'R²': r2
    }

    #print results
    print(f"  Benchmark: {name}")
    print(f"  Test MAPE: {mape:.2f}%")
    print(f"  Test R²: {r2:.4f}")
    print(f"  Test MAE: {mae:.4f}")
    print(f"  Test RMSE: {rmse:.4f}")
    print("-" * 40)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

#define the earlystopping callback
early_stopping_callback = EarlyStopping(
    monitor="val_loss",           #monitor loss
    min_delta=0,                  #minimum change to qualify as an improvement
    patience=5,                   #number of epochs to wait for improvement before stopping
    verbose=1,                    #print when stopping early
    mode="min",                   # 'min' because we want to minimize the loss
    restore_best_weights=True,    #restore weights from the best epoch when stopping
)

# save the best model
checkpoint_callback = ModelCheckpoint(
    'best_model.keras',   #path to save the best model weights
    monitor='val_loss',        #monitor loss
    save_best_only=True,       #save only the best weights
    mode='min',                #'min' because we want to minimize the loss
    verbose=1                  #print when saving the best weights
)

# uncertainty part 1: dropout layers

In [ ]:
def build_mc_dropout_model(input_steps, feature_count, dropout_rate=0.1):
    inputs = Input(shape=(input_steps, feature_count))
    
    # LSTM layers with dropout, return_sequence for next layer, dropout during training, and in the recurrent part as well
    x = LSTM(128, activation='tanh', return_sequences=True, dropout=dropout_rate, recurrent_dropout=dropout_rate)(inputs)
    x = LSTM(64, activation='tanh', return_sequences=True, dropout=dropout_rate, recurrent_dropout=dropout_rate)(inputs)
    x = LSTM(32, activation='tanh')(x)
    
    # ensure dropout is always applied, during training and predicting
    x = Lambda(lambda x: tf.keras.backend.dropout(x, level=dropout_rate))(x)
    
    outputs = Dense(1)(x)
    
    model = Model(inputs, outputs)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='mse', metrics=['mae'])
    
    return model

#build model
model_uncertain1 = build_mc_dropout_model(input_steps, len(features))

In [ ]:
#train the model
history = model_uncertain1.fit(all_train_X, all_train_y, validation_data=(all_val_X, all_val_y), epochs=20, batch_size=64, verbose=1, callbacks=[checkpoint_callback])

In [ ]:
#function to perform MC Dropout inference
def mc_dropout_predictions(model, X_input, num_samples=100):
    """
    Perform Monte Carlo Dropout sampling on a trained model.

    Args:
        model: Trained Keras model with dropout layers.
        X_input: Input data for prediction.
        num_samples: Number of stochastic forward passes.

    Returns:
        mean_prediction: Mean of sampled predictions.
        std_prediction: Standard deviation of sampled predictions (uncertainty estimate).
    """
    f_passes = []
    for i in range(num_samples):
        #perform a forward pass with dropout enabled
        sample = model(X_input, training=True).numpy().flatten()
        f_passes.append(sample)
        
        #Print progress every 10 iterations
        #if (i + 1) % 10 == 0 or i == num_samples - 1:
        #    print(f"MC Dropout sampling: {i + 1}/{num_samples} iterations completed.")
    
    f_passes = np.array(f_passes)
    mean_prediction = f_passes.mean(axis=0)
    std_prediction = f_passes.std(axis=0)
    
    return mean_prediction, std_prediction

#run MC Dropout sampling
num_samples = 100
mean_pred, std_pred = mc_dropout_predictions(model_uncertain1, all_test_X, num_samples=num_samples)

#compute 95% confidence interval
lower_bound = mean_pred - 1.645 * std_pred
upper_bound = mean_pred + 1.645 * std_pred

In [ ]:
all_test_y_flat = all_test_y.reshape(-1)  #ensure it's a 1D array
all_test_y_indices_flat = all_test_y_indices.reshape(-1)  #flatten indices to match

df_test = pd.DataFrame({
    "actual": all_test_y_flat,
}, index=pd.Index(all_test_y_indices_flat, name="timestamp"))

#create a DataFrame for plotting
df_results = pd.DataFrame({
    "timestamp": df_test.index,
    "predicted_mean": mean_pred,
    "lower_bound": lower_bound,
    "upper_bound": upper_bound,
    "actual": all_test_y.flatten()
})

fig = go.Figure()

#add actual values trace
fig.add_trace(go.Scatter(
    x=df_results["timestamp"], 
    y=df_results["actual"],
    mode='lines', 
    name='Actual',
    line=dict(color='black')
))

#add predicted mean trace
fig.add_trace(go.Scatter(
    x=df_results["timestamp"], 
    y=df_results["predicted_mean"],
    mode='lines', 
    name='Predicted Mean',
    line=dict(color='blue')
))

#add lower bound trace (invisible line)
fig.add_trace(go.Scatter(
    x=df_results["timestamp"], 
    y=df_results["lower_bound"],
    mode='lines',
    name='Lower Bound',
    line=dict(color='rgba(0,0,255,0)'),
    showlegend=False
))

#add upper bound trace with fill to the previous trace (lower bound)
fig.add_trace(go.Scatter(
    x=df_results["timestamp"],
    y=df_results["upper_bound"],
    mode='lines',
    name='95% Confidence Interval',
    line=dict(color='rgba(0,0,255,0)'),
    fill='tonexty',  #fills between this trace and the previous (lower bound) trace
    fillcolor='rgba(0,0,255,0.2)'
))

# Update layout
fig.update_layout(
    title='Solar Power Forecasting with MC Dropout Uncertainty',
    xaxis_title='Time',
    yaxis_title='Power Output (W)',
    xaxis_rangeslider_visible=True
)

fig.show()

In [ ]:
df_results.set_index("timestamp", inplace=True)
df_results

# denormalize

In [ ]:
df_results = df_results.reset_index()
df_results['timestamp'] = pd.to_datetime(df_results['timestamp'])
df_results['timestamp'] = df_results['timestamp'].dt.tz_localize('UTC')
df_results.set_index('timestamp', inplace=True)


#bring back to normal data
#read in the normalization profile factors

#merge the two DataFrames on the 'time' index
df_merged_all = df_results.merge(df_data[['adjusted_P_max', 'mean_actualPowerTot_W_inverter']], left_index=True, right_index=True, how='left')
#df_merged_all = df_results.merge(df_data[['mean_actualPowerTot_W_inverter']], left_index=True, right_index=True, how='left')
#lose some data from full_adjusted_df near end since the df_data doesn't have full last day

#check for missing values (NaN) in adjusted_P_max
if df_merged_all['adjusted_P_max'].isna().any():
    print("Warning: Some values are missing in the normalization profile.")
    
df_merged_all = df_merged_all[(df_merged_all["predicted_mean"] > 0) & (df_merged_all["predicted_mean"] < 5)]

#denormalize the 'mean_actualPowerTot_W_inverter' column by multiplying by the 'adjusted_P_max' column
df_merged_all['denormalized_value_predicted'] = df_merged_all['predicted_mean'] * df_merged_all['adjusted_P_max']
df_merged_all['denormalized_lower_bound'] = df_merged_all['lower_bound'] * df_merged_all['adjusted_P_max']
df_merged_all['denormalized_upper_bound'] = df_merged_all['upper_bound'] * df_merged_all['adjusted_P_max']


df_merged_all

In [ ]:
fig = go.Figure()

#add actual values trace
fig.add_trace(go.Scatter(
    x=df_merged_all.index, 
    y=df_merged_all["mean_actualPowerTot_W_inverter"],
    mode='lines', 
    name='Actual',
    line=dict(color='black')
))

#add predicted mean trace
fig.add_trace(go.Scatter(
    x=df_merged_all.index, 
    y=df_merged_all["denormalized_value_predicted"],
    mode='lines', 
    name='Predicted Mean',
    line=dict(color='blue')
))

#add lower bound trace (invisible line)
fig.add_trace(go.Scatter(
    x=df_merged_all.index, 
    y=df_merged_all["denormalized_lower_bound"],
    mode='lines',
    name='Lower Bound',
    line=dict(color='rgba(0,0,255,0)'),
    showlegend=False
))

#add upper bound trace with fill to the previous trace (lower bound)
fig.add_trace(go.Scatter(
    x=df_merged_all.index,
    y=df_merged_all["denormalized_upper_bound"],
    mode='lines',
    name='95% Confidence Interval',
    line=dict(color='rgba(0,0,255,0)'),
    fill='tonexty',  #fills between this trace and the previous (lower bound) trace
    fillcolor='rgba(0,0,255,0.2)'
))

#update layout
fig.update_layout(
    title='Solar Power Forecasting with MC Dropout Uncertainty',
    xaxis_title='Time',
    yaxis_title='Power Output (W)',
    xaxis_rangeslider_visible=True
)

fig.show()

In [ ]:
evaluate_benchmark("uncertainty 1 min", df_merged_all['mean_actualPowerTot_W_inverter'] ,df_merged_all["denormalized_value_predicted"])

# uncertainty 1 with the testing at every timestamp

In [ ]:
#function to compute evaluation metrics
def evaluate_benchmark2(y_true, y_pred):
    """Computes MAE, RMSE, MAPE, and R² and stores in a dictionary."""
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    
    #avoid zero division in MAPE
    valid_mask = y_true != 0  
    mape = np.mean(np.abs((y_true[valid_mask] - y_pred[valid_mask]) / y_true[valid_mask])) * 100  
    r2 = r2_score(y_true, y_pred)

    results = {
        "MAPE": round(mape, 2),
        "R²": round(r2, 4),
        "MAE": round(mae, 4),
        "RMSE": round(rmse, 4),
    }

    return results

In [ ]:
#set the number of hours to predict ahead
prediction_horizon = 24  #number of hours to predict ahead

all_predictions = []
all_predictions_recursive = []
all_predictions_indices = []


all_predictions_lower = []
all_predictions_upper = []

steps = 24

print(f"Total number of test sets at stepsize {steps}: {len(all_test_X)// steps}")

#loop over the entire test set
amount_of_times = 0
for test_idx in range(0, len(all_test_X), steps):  #iterate over each test example
    starting_window = all_test_X[test_idx]
    starting_window_indeces = all_test_X_indices[test_idx]
    #print(starting_window_indeces)

    input_window = starting_window
    input_window_indices = starting_window_indeces

    predictions_recursive = []
    predictions_indices = []
    predictions_lower = []
    predictions_upper = []

    for i in range(prediction_horizon * 4):  #total steps in 15minutes intervals
        #predict next minute
        num_samples = 10
        mean_pred, std_pred = mc_dropout_predictions(model_uncertain1, input_window[np.newaxis, :, :], num_samples=num_samples)
        #print(mean_pred)
        #compute 95% confidence interval
        lower_bound = mean_pred - 1.645 * std_pred
        upper_bound = mean_pred + 1.645 * std_pred
        
        prediction = mean_pred
        predictions_recursive.append(prediction[0])
        predictions_lower.append(lower_bound[0])
        predictions_upper.append(upper_bound[0])


        #update time indices for predictions
        next_index = input_window_indices[-1] + pd.Timedelta(minutes=15)
        predictions_indices.append(next_index)

        #update the input window for the next step (shift window to the left)
        input_window = np.roll(input_window, -1, axis=0)
        
        #update the last element of the window with the prediction
        input_window[-1, 0] = prediction[0]
        
        #update dayofweek
        input_window[-1, 1] = next_index.dayofweek
        input_window[-1, 2] = next_index.month

        #update hour sin and cos
        next_hour = next_index.hour
        input_window[-1, 3] = np.sin(2 * np.pi * next_hour / 24)
        input_window[-1, 4] = np.cos(2 * np.pi * next_hour / 24)
        
        next_minute = next_index.minute

        # Encode the 15-minute intervals within an hour
        input_window[-1, 5] = np.sin(2 * np.pi * next_minute / 60)
        input_window[-1, 6] = np.cos(2 * np.pi * next_minute / 60)
        
        next_minute = next_index.minute
        input_window[-1, 7] = np.sin(2 * np.pi * next_minute / 60)
        input_window[-1, 8] = np.cos(2 * np.pi * next_minute / 60)
        
        
        ##################################################
        #test future data as direct insert in weather
        
        
        #add the other features from the previous day
        #next_index_previous_day = next_index - pd.Timedelta(days=1)
        
        #for daylight savings, timestamps not present
        # Check if the exact timestamp exists in the index
        #if next_index_previous_day in df_data.index:
        #    data_next_day = df_data.loc[next_index_previous_day]
        #else:
        #    #make next_index_previous_day also timezone aware
        #    next_index_previous_day = next_index_previous_day.tz_localize(df_data.index.tzinfo)
        #    # If not, get the next closest available timestamp (forward direction)
        #    closest_timestamp_idx = df_data.index.searchsorted(next_index_previous_day, side='right')
        #    closest_timestamp = df_data.index[closest_timestamp_idx] if closest_timestamp_idx < len(df_data) else df_data.index[-1]

        #    data_next_day = df_data.loc[closest_timestamp]
        
        if next_index in df_data.index:
            data_next_day = df_data.loc[next_index]
        else:
            next_index = next_index.tz_localize(df_data.index.tzinfo)
            closest_timestamp_idx = df_data.index.searchsorted(next_index, side='right')
            closest_timestamp = df_data.index[closest_timestamp_idx] if closest_timestamp_idx < len(df_data) else df_data.index[-1]

            data_next_day = df_data.loc[closest_timestamp]
            
        
        #######################################################
        
        if isinstance(data_next_day, pd.Series):
            data_next_day = data_next_day.to_frame().T
            
        try:
            input_window[-1, 9] = data_next_day.iloc[0]["temp_dry_shelter_avg"]
            input_window[-1, 10] = data_next_day.iloc[0]["normalized_value_shift_24"]
            input_window[-1, 11] = data_next_day.iloc[0]["diffuseIrradiance_Wpm2"]
            input_window[-1, 12] = data_next_day.iloc[0]["directNormalIrradiance_Wpm2"]
            input_window[-1, 13] = data_next_day.iloc[0]["globalHorizontalIrradiance_Wpm2"]
            
            input_window[-1, 14] = data_next_day.iloc[0]["precip_quantity"]
            input_window[-1, 15] = data_next_day.iloc[0]["temp_soil_avg"]
            input_window[-1, 16] = data_next_day.iloc[0]["wind_direction"]
            input_window[-1, 17] = data_next_day.iloc[0]["wind_gusts_speed"]
            input_window[-1, 18] = data_next_day.iloc[0]["humidity_rel_shelter_avg"]
            input_window[-1, 19] = data_next_day.iloc[0]["pressure"]
            input_window[-1, 20] = data_next_day.iloc[0]["sun_duration"]
            input_window[-1, 21] = data_next_day.iloc[0]["short_wave_from_sky_avg"]
            input_window[-1, 22] = data_next_day.iloc[0]["sun_int_avg"]
            input_window[-1, 23] = data_next_day.iloc[0]["wind_speed"]
            
            #input_window[-1, 24] = data_next_day.iloc[0]["precip_quantity_future"]
            #input_window[-1, 25] = data_next_day.iloc[0]["temp_soil_avg_future"]
            #input_window[-1, 26] = data_next_day.iloc[0]["wind_direction_future"]
            #input_window[-1, 27] = data_next_day.iloc[0]["wind_gusts_speed_future"]
            #input_window[-1, 28] = data_next_day.iloc[0]["humidity_rel_shelter_avg_future"]
            #input_window[-1, 29] = data_next_day.iloc[0]["pressure_future"]
            #input_window[-1, 30] = data_next_day.iloc[0]["sun_duration_future"]
            #input_window[-1, 31] = data_next_day.iloc[0]["short_wave_from_sky_avg_future"]
            #input_window[-1, 32] = data_next_day.iloc[0]["sun_int_avg_future"]
            #input_window[-1, 33] = data_next_day.iloc[0]["wind_speed_future"]
            
            
        except KeyError:
            input_window[-1, 9] = np.nan
            input_window[-1, 10] = np.nan
            input_window[-1, 11] = np.nan
            input_window[-1, 12] = np.nan
            input_window[-1, 13] = np.nan
            
            input_window[-1, 14] = np.nan
            input_window[-1, 15] = np.nan
            input_window[-1, 16] = np.nan
            input_window[-1, 17] = np.nan
            input_window[-1, 18] = np.nan
            input_window[-1, 19] = np.nan
            input_window[-1, 20] = np.nan
            input_window[-1, 21] = np.nan
            input_window[-1, 22] = np.nan
            input_window[-1, 23] = np.nan
            
            #input_window[-1, 24] = np.nan
            #input_window[-1, 25] = np.nan
            #input_window[-1, 26] = np.nan
            #input_window[-1, 27] = np.nan
            #input_window[-1, 28] = np.nan
            #input_window[-1, 29] = np.nan
            #input_window[-1, 30] = np.nan
            #input_window[-1, 31] = np.nan
            #input_window[-1, 32] = np.nan
            #input_window[-1, 33] = np.nan
            
        #update indices
        input_window_indices = np.roll(input_window_indices, -1, axis=0)
        input_window_indices[-1] = next_index
            
        # print progress for every 100 steps or at the last step
        if i % 10 == 0 or i == prediction_horizon * 4 - 1:
            print(f"Prediction progress for test set {test_idx//steps + 1}: Step {i + 1} / {prediction_horizon * 4} ({(i + 1) / (prediction_horizon * 4) * 100:.2f}%)")

        
    
    #convert predictions and indices to numpy arrays
    predictions_recursive = np.array(predictions_recursive)
    predictions_indices = np.array(predictions_indices)

    #store predictions for this test example
    all_predictions_recursive.append(predictions_recursive)
    all_predictions_indices.append(predictions_indices)
    all_predictions_upper.append(predictions_upper)
    all_predictions_lower.append(predictions_lower)

    #store predictions for this test example
    #add in individual dataframe
    df_result = pd.DataFrame({
        "predicted_value": predictions_recursive,
        "lower_bound": predictions_lower,
        "upper_bound": predictions_upper,
    }, index=pd.Index(predictions_indices, name="timestamp"))

        
    all_predictions.append(df_result)
    
    print("amount_of_times: ", amount_of_times)
    
    amount_of_times += 1
    
    #if(amount_of_times >= 1000):
    #    break

print("Prediction completed for all test samples.")

In [ ]:
test = all_predictions.copy()

for i, df in enumerate(test):
    df_merged_all = []
    
    df = df.reset_index()
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['timestamp'] = df['timestamp'].dt.tz_localize('UTC')
    df.set_index('timestamp', inplace=True)
    
    #merge the two DataFrames on the 'time' index
    df_merged_all = df.merge(df_data[['adjusted_P_max', 'mean_actualPowerTot_W_inverter']], left_index=True, right_index=True, how='left')
    #check for missing values (NaN) in adjusted_P_max
    if df_merged_all['adjusted_P_max'].isna().any():
        print("Warning: Some values are missing in the normalization profile.")

    #denormalize the 'mean_actualPowerTot_W_inverter' column by multiplying by the 'adjusted_P_max' column
    df_merged_all['denormalized_value_predicted'] = df_merged_all['predicted_value'] * df_merged_all['adjusted_P_max']
    df_merged_all['denormalized_lower_bound'] = df_merged_all['lower_bound'] * df_merged_all['adjusted_P_max']
    df_merged_all['denormalized_upper_bound'] = df_merged_all['upper_bound'] * df_merged_all['adjusted_P_max']

    test[i] = df_merged_all
    
test

In [ ]:
for i, df in enumerate(test):
    before = len(df)
    after = len(df.dropna())
    print(f"test[{i}]: {before - after} rows dropped due to NaNs")
    test[i] = df.dropna()

In [ ]:
all_results = []

for df in test:
    result = evaluate_benchmark2(df['mean_actualPowerTot_W_inverter'], df['denormalized_value_predicted'])
    
    within_interval = ((df['mean_actualPowerTot_W_inverter'] >= df['denormalized_lower_bound']) &
                   (df['mean_actualPowerTot_W_inverter'] <= df['denormalized_upper_bound']))
    CP = within_interval.mean()

    interval_width = df['denormalized_upper_bound'] - df['denormalized_lower_bound']
    MIW = interval_width.mean()
    relative_interval_width = interval_width / df['mean_actualPowerTot_W_inverter']
    rMIW = relative_interval_width.mean()
    
    result["CP"] = CP
    result["MIW"] = MIW
    result["rMIW"] = rMIW
    
    all_results.append(result)

#compute average metrics across all DataFrames
average_results = {
    "Benchmark": "Average Across All",
    "MAPE": round(np.mean([r["MAPE"] for r in all_results]), 2),
    "R²": round(np.mean([r["R²"] for r in all_results]), 4),
    "MAE": round(np.mean([r["MAE"] for r in all_results]), 4),
    "RMSE": round(np.mean([r["RMSE"] for r in all_results]), 4),
    "CP": round(np.mean([r["CP"] for r in all_results]), 4),
    "MIW": round(np.mean([r["MIW"] for r in all_results]), 4),
    "rMIW": round(np.mean([r["rMIW"] for r in all_results]), 4),
}

#print all individual results and the average
for i, res in enumerate(all_results):
    print(f"Results for DataFrame {i+1}: {res}")
print("-" * 40)
print("Final Average Results:", average_results)

In [ ]:
dataframe = test[100]

fig = go.Figure()

#add actual values trace
fig.add_trace(go.Scatter(
    x=dataframe.index, 
    y=dataframe["mean_actualPowerTot_W_inverter"],
    mode='lines', 
    name='Actual',
    line=dict(color='black')
))

#add predicted mean trace
fig.add_trace(go.Scatter(
    x=dataframe.index, 
    y=dataframe["denormalized_value_predicted"],
    mode='lines', 
    name='Predicted Mean',
    line=dict(color='blue')
))

#add lower bound trace (invisible line)
fig.add_trace(go.Scatter(
    x=dataframe.index, 
    y=dataframe["denormalized_lower_bound"],
    mode='lines',
    name='Lower Bound',
    line=dict(color='rgba(0,0,255,0)'),
    showlegend=False
))

#add upper bound trace with fill to the previous trace (lower bound)
fig.add_trace(go.Scatter(
    x=dataframe.index,
    y=dataframe["denormalized_upper_bound"],
    mode='lines',
    name='95% Confidence Interval',
    line=dict(color='rgba(0,0,255,0)'),
    fill='tonexty',  #fills between this trace and the previous (lower bound) trace
    fillcolor='rgba(0,0,255,0.2)'
))

#update layout
fig.update_layout(
    title='Solar Power Forecasting with MC Dropout Uncertainty',
    xaxis_title='Time',
    yaxis_title='Power Output (W)',
    xaxis_rangeslider_visible=True
)

fig.show()

In [ ]:
output_steps_day = 96
#initialize list of DataFrames for each time step (96 DataFrames for 96 time steps)
time_step_dfs = {i: pd.DataFrame(columns=['actual', 'predicted']) for i in range(output_steps_day)}

#loop over each DataFrame in the test dataset and populate the 96 DataFrames
for df in test:
    #get the 'actual' and 'predicted' values from the DataFrame
    actual_values = df['mean_actualPowerTot_W_inverter'].values
    predicted_values = df['denormalized_value_predicted'].values

    #populate the DataFrames corresponding to each time step
    for i in range(output_steps_day):  #we have 96 time steps (from 0 to 95)
        #create a temporary dataFrame to hold the current actual and predicted values
        temp_df = pd.DataFrame({'actual': [actual_values[i] if i < len(actual_values) else np.nan],
                                'predicted': [predicted_values[i] if i < len(predicted_values) else np.nan]})
        
        time_step_dfs[i] = pd.concat([time_step_dfs[i], temp_df], ignore_index=True)

#evaluate each DataFrame individually using the evaluate_benchmark2 function
all_results = []

for i in range(output_steps_day):
    #get the actual and predicted values for the current time step DataFrame
    step_df = time_step_dfs[i]

    #drop any rows with NaN values
    step_df = step_df.dropna()

    result = evaluate_benchmark2(step_df['actual'], step_df['predicted'])
    result["Time Step"] = i
    
    all_results.append(result)

#compute the average results across all time steps
average_results = {
    "MAPE": round(np.mean([r["MAPE"] for r in all_results]), 2),
    "R²": round(np.mean([r["R²"] for r in all_results]), 4),
    "MAE": round(np.mean([r["MAE"] for r in all_results]), 4),
    "RMSE": round(np.mean([r["RMSE"] for r in all_results]), 4),
}

for res in all_results:
    print(f"Results for Time Step {res['Time Step']}: {res}")

print("-" * 40)
print("Final Average Results:", average_results)

In [ ]:
#extract the metrics for plotting
time_steps = [r["Time Step"] for r in all_results]
mape_values = [r["MAPE"] for r in all_results]
mae_values = [r["MAE"] for r in all_results]
rmse_values = [r["RMSE"] for r in all_results]
r2_values = [r["R²"] for r in all_results]

#create subplots
fig = go.Figure()

#plot MAPE
#fig.add_trace(go.Scatter(x=time_steps, y=mape_values, mode='lines', name='MAPE', line=dict(color='blue')))

#plot MAE
fig.add_trace(go.Scatter(x=time_steps, y=mae_values, mode='lines', name='MAE', line=dict(color='green')))

#plot RMSE
#fig.add_trace(go.Scatter(x=time_steps, y=rmse_values, mode='lines', name='RMSE', line=dict(color='red')))

#plot R²
#fig.add_trace(go.Scatter(x=time_steps, y=r2_values, mode='lines', name='R²', line=dict(color='orange')))

fig.update_layout(
    title="Evaluation Metrics Across Time Steps",
    xaxis_title="Time Step",
    yaxis_title="Metric Value",
    legend_title="Metrics"
)

fig.show()

file_path = r"C:\Users\samr0\OneDrive - KU Leuven\Documents\!School\master\Thesis\data\timestep_results_MC_recursive_test7_fixed.csv"
df_results = pd.DataFrame(all_results)
df_results.to_csv(file_path, index=False)

# uncertainty quantiles:

In [ ]:
def quantile_loss(q):
    def loss(y_true, y_pred):
        e = y_true - y_pred
        return tf.reduce_mean(tf.maximum(q * e, (q - 1) * e))
    return loss

In [ ]:
def build_quantile_model(input_steps, feature_count, quantiles=[0.05, 0.5, 0.95]):
    """
    Builds a Keras model that predicts multiple quantiles at once.
    """
    inputs = layers.Input(shape=(input_steps, feature_count))
    
    x = layers.LSTM(128, return_sequences=True)(inputs)
    x = layers.LSTM(64, return_sequences=True)(x)
    x = layers.LSTM(32)(x)
    
    #separate Dense layers for each quantile
    outputs = [layers.Dense(1, name=f'quantile_{q}')(x) for q in quantiles]
    
    model = Model(inputs, outputs)
    
    return model

In [ ]:
quantiles = [0.05, 0.5, 0.95]

#build the model
model_quantile = build_quantile_model(input_steps, feature_count=len(features), quantiles=quantiles)

#compile the model using a list of losses
model_quantile.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss=[quantile_loss(q) for q in quantiles]  #list of separate loss functions
)

#reshape training targets to match multiple outputs
all_train_y_list = [all_train_y] * len(quantiles)
all_val_y_list = [all_val_y] * len(quantiles)

#train the model
history = model_quantile.fit(
    all_train_X, all_train_y_list,  # Pass list of targets
    validation_data=(all_val_X, all_val_y_list),
    epochs=20,
    batch_size=32
)

In [ ]:
y_pred_list = model_quantile.predict(all_test_X)  #returns [array_q05, array_q50, array_q95]

#extract each quantile prediction
y_pred_q05 = y_pred_list[0].flatten()  # 5th percentile
y_pred_q50 = y_pred_list[1].flatten()  # 50th percentile (median)
y_pred_q95 = y_pred_list[2].flatten()  # 95th percentile

In [ ]:
all_test_y_flat = all_test_y.reshape(-1)  #ensure it's a 1D array
all_test_y_indices_flat = all_test_y_indices.reshape(-1)  #flatten indices to match

df_results = pd.DataFrame({
    'q05': y_pred_q05,
    'q50': y_pred_q50,
    'q95': y_pred_q95,
    'actual': all_test_y_flat
}, index=pd.Index(all_test_y_indices_flat, name="timestamp"))


fig = go.Figure()

# Actual
fig.add_trace(go.Scatter(
    x=df_results.index, 
    y=df_results['actual'],
    mode='lines', 
    name='Actual', 
    line=dict(color='black')
))

# Median (q50)
fig.add_trace(go.Scatter(
    x=df_results.index, 
    y=df_results['q50'],
    mode='lines', 
    name='Predicted Median (q50)',
    line=dict(color='blue')
))

# Lower quantile (q05) - invisible line
fig.add_trace(go.Scatter(
    x=df_results.index, 
    y=df_results['q05'],
    mode='lines', 
    name='q05',
    line=dict(color='rgba(0,0,255,0)'),
    showlegend=False
))

# Upper quantile (q95) - fill between q05 and q95
fig.add_trace(go.Scatter(
    x=df_results.index, 
    y=df_results['q95'],
    mode='lines',
    name='95% Interval',
    line=dict(color='rgba(0,0,255,0)'),
    fill='tonexty',  # fill between this trace and the previous (q05)
    fillcolor='rgba(0,0,255,0.2)'
))

fig.update_layout(
    title='Quantile Forecasting with LSTM',
    xaxis_title='Time',
    yaxis_title='Value',
    xaxis_rangeslider_visible=True
)

fig.show()


In [ ]:
df_results

# denormalize

In [ ]:
df_results.reset_index(inplace=True)
df_results['timestamp'] = pd.to_datetime(df_results['timestamp'])
df_results['timestamp'] = df_results['timestamp'].dt.tz_localize ('UTC')
df_results.set_index('timestamp', inplace=True)

#bring back to normal data
#read in the normalization profile factors

#merge the two DataFrames on the 'time' index
df_merged_all = df_results.merge(df_data[['adjusted_P_max', 'mean_actualPowerTot_W_inverter']], left_index=True, right_index=True, how='left')
#lose some data from full_adjusted_df near end since the df_data doesn't have full last day

#check for missing values (NaN) in adjusted_P_max
if df_merged_all['adjusted_P_max'].isna().any():
    print("Warning: Some values are missing in the normalization profile.")
    
# currently cut of a part that goes infinite

#df_merged_all = df_merged_all[(df_merged_all["predicted_mean"] > 0) & (df_merged_all["predicted_mean"] < 5)]

#denormalize the 'mean_actualPowerTot_W_inverter' column by multiplying by the 'adjusted_P_max' column
df_merged_all['denormalized_q05'] = df_merged_all['q05'] * df_merged_all['adjusted_P_max']
df_merged_all['denormalized_q50'] = df_merged_all['q50'] * df_merged_all['adjusted_P_max']
df_merged_all['denormalized_q95'] = df_merged_all['q95'] * df_merged_all['adjusted_P_max']


df_merged_all

In [ ]:
evaluate_benchmark("uncertainty quantile 1 timestep", df_merged_all['mean_actualPowerTot_W_inverter'] ,df_merged_all["denormalized_q50"])

# quantile testing on every testset

In [ ]:
#function to compute evaluation metrics
def evaluate_benchmark2(y_true, y_pred):
    """Computes MAE, RMSE, MAPE, and R² and stores in a dictionary."""
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    
    #avoid zero division in MAPE
    valid_mask = y_true != 0  
    mape = np.mean(np.abs((y_true[valid_mask] - y_pred[valid_mask]) / y_true[valid_mask])) * 100  
    r2 = r2_score(y_true, y_pred)
    
    #relative mae
    avg = np.mean(y_true[valid_mask])
    rel_mae = mae/avg
    rel_rmse = rmse/avg

    results = {
        "MAPE": round(mape, 2),
        "R²": round(r2, 4),
        "MAE": round(mae, 4),
        "RMSE": round(rmse, 4),
        "rel_MAE": round(rel_mae, 4),
        "rel_RMSE": round(rel_rmse, 4),
    }

    return results

In [ ]:
#set the number of hours to predict ahead
prediction_horizon = 24  #number of hours to predict ahead

#initialize lists to store predictions for the whole test set
all_predictions_recursive = []
all_predictions_indices = []

all_predictions_q05 = []
all_predictions_q95 = []

#for the testing of every testset
all_predictions = []

#loop over the entire test set
amount_of_times = 0

steps = 8

print(f"Total number of test sets at stepsize {steps}: {len(all_test_X) // steps}")

for test_idx in range(0, len(all_test_X), steps):  #iterate over each test example
    starting_window = all_test_X[test_idx]
    starting_window_indeces = all_test_X_indices[test_idx]
    #print(starting_window_indeces)

    input_window = starting_window
    input_window_indices = starting_window_indeces

    predictions_recursive = []
    predictions_indices = []
    predictions_q05 = []
    predictions_q95 = []
    
    for i in range(prediction_horizon * 4):  #total steps in 15minutes intervals
        #predict next minute
        num_samples = 20
        y_pred_list = model_quantile.predict(input_window[np.newaxis, :, :])  # Returns [array_q05, array_q50, array_q95]

        # Extract each quantile prediction
        y_pred_q05 = y_pred_list[0].flatten()[0]  # 5th percentile
        y_pred_q50 = y_pred_list[1].flatten()[0]  # 50th percentile (median)
        y_pred_q95 = y_pred_list[2].flatten()[0]  # 95th percentile
        
        predictions_recursive.append(y_pred_q50)
        predictions_q05.append(y_pred_q05)
        predictions_q95.append(y_pred_q95)

        #update time indices for predictions
        next_index = input_window_indices[-1] + pd.Timedelta(minutes=15)
        predictions_indices.append(next_index)

        #update the input window for the next step (shift window to the left)
        input_window = np.roll(input_window, -1, axis=0)
        
        #update the last element of the window with the prediction
        input_window[-1, 0] = y_pred_q50
        
        #input_window[-1, 1] = next_index.day_of_week
        #input_window[-1, 2] = next_index.month

        #update hour sin and cos
        next_hour = next_index.hour
        #input_window[-1, 3] = np.sin(2 * np.pi * next_hour / 24)
        #input_window[-1, 4] = np.cos(2 * np.pi * next_hour / 24)
        
        
        next_day = next_index.day
        #input_window[-1, 5] = np.sin(2 * np.pi * next_day / 365)
        #input_window[-1, 6] = np.cos(2 * np.pi * next_day / 365)
        
        next_minute = next_index.minute
        #input_window[-1, 7] = np.sin(2 * np.pi * next_minute / 60)
        #input_window[-1, 8] = np.cos(2 * np.pi * next_minute / 60)
        
        
        ##################################################
        #test future data as direct insert in weather
        
        
        #add the other features from the previous day
        next_index_previous_day = next_index - pd.Timedelta(days=1)
        
        #for daylight savings, timestamps not present
        # Check if the exact timestamp exists in the index
        if next_index_previous_day in df_data.index:
            data_next_day = df_data.loc[next_index_previous_day]
        else:
            #make next_index_previous_day also timezone aware
            next_index_previous_day = next_index_previous_day.tz_localize(df_data.index.tzinfo)
            # If not, get the next closest available timestamp (forward direction)
            closest_timestamp_idx = df_data.index.searchsorted(next_index_previous_day, side='right')
            closest_timestamp = df_data.index[closest_timestamp_idx] if closest_timestamp_idx < len(df_data) else df_data.index[-1]

            data_next_day = df_data.loc[closest_timestamp]
        
        #if next_index in df_data.index:
        #    data_next_day = df_data.loc[next_index]
        #else:
        #    next_index = next_index.tz_localize(df_data.index.tzinfo)
        #    closest_timestamp_idx = df_data.index.searchsorted(next_index, side='right')
        #    closest_timestamp = df_data.index[closest_timestamp_idx] if closest_timestamp_idx < len(df_data) else df_data.index[-1]

        #    data_next_day = df_data.loc[closest_timestamp]
            
        
        #######################################################
        
        
        
        if isinstance(data_next_day, pd.Series):
            data_next_day = data_next_day.to_frame().T
            
        try:
            input_window[-1, 1] = data_next_day.iloc[0]["temp_dry_shelter_avg"]
            input_window[-1, 2] = data_next_day.iloc[0]["normalized_value_shift_24"]
            input_window[-1, 3] = data_next_day.iloc[0]["diffuseIrradiance_Wpm2"]
            input_window[-1, 4] = data_next_day.iloc[0]["directNormalIrradiance_Wpm2"]
            input_window[-1, 5] = data_next_day.iloc[0]["globalHorizontalIrradiance_Wpm2"]
            
            #input_window[-1, 14] = data_next_day.iloc[0]["precip_quantity"]
            #input_window[-1, 15] = data_next_day.iloc[0]["temp_soil_avg"]
            #input_window[-1, 16] = data_next_day.iloc[0]["wind_direction"]
            #input_window[-1, 17] = data_next_day.iloc[0]["wind_gusts_speed"]
            #input_window[-1, 18] = data_next_day.iloc[0]["humidity_rel_shelter_avg"]
            #input_window[-1, 19] = data_next_day.iloc[0]["pressure"]
            #input_window[-1, 20] = data_next_day.iloc[0]["sun_duration"]
            #input_window[-1, 21] = data_next_day.iloc[0]["short_wave_from_sky_avg"]
            #input_window[-1, 22] = data_next_day.iloc[0]["sun_int_avg"]
            #input_window[-1, 23] = data_next_day.iloc[0]["wind_speed"]
            
            #input_window[-1, 24] = data_next_day.iloc[0]["precip_quantity_future"]
            #input_window[-1, 25] = data_next_day.iloc[0]["temp_soil_avg_future"]
            #input_window[-1, 26] = data_next_day.iloc[0]["wind_direction_future"]
            #input_window[-1, 27] = data_next_day.iloc[0]["wind_gusts_speed_future"]
            #input_window[-1, 28] = data_next_day.iloc[0]["humidity_rel_shelter_avg_future"]
            #input_window[-1, 29] = data_next_day.iloc[0]["pressure_future"]
            #input_window[-1, 30] = data_next_day.iloc[0]["sun_duration_future"]
            #input_window[-1, 31] = data_next_day.iloc[0]["short_wave_from_sky_avg_future"]
            #input_window[-1, 32] = data_next_day.iloc[0]["sun_int_avg_future"]
            #input_window[-1, 33] = data_next_day.iloc[0]["wind_speed_future"]
            
            
        except KeyError:
            input_window[-1, 1] = np.nan
            input_window[-1, 2] = np.nan
            input_window[-1, 3] = np.nan
            input_window[-1, 4] = np.nan
            input_window[-1, 5] = np.nan
            
            #input_window[-1, 14] = np.nan
            #input_window[-1, 15] = np.nan
            #input_window[-1, 16] = np.nan
            #input_window[-1, 17] = np.nan
            #input_window[-1, 18] = np.nan
            #input_window[-1, 19] = np.nan
            #input_window[-1, 20] = np.nan
            #input_window[-1, 21] = np.nan
            #input_window[-1, 22] = np.nan
            #input_window[-1, 23] = np.nan
            
            #input_window[-1, 24] = np.nan
            #input_window[-1, 25] = np.nan
            #input_window[-1, 26] = np.nan
            #input_window[-1, 27] = np.nan
            #input_window[-1, 28] = np.nan
            #input_window[-1, 29] = np.nan
            #input_window[-1, 30] = np.nan
            #input_window[-1, 31] = np.nan
            #input_window[-1, 32] = np.nan
            #input_window[-1, 33] = np.nan

        #update indices
        input_window_indices = np.roll(input_window_indices, -1, axis=0)
        input_window_indices[-1] = next_index

        # print progress for every 100 steps or at the last step
        if i % 10 == 0 or i == prediction_horizon * 4 - 1:
            print(f"Prediction progress for test set {test_idx//steps + 1}: Step {i + 1} / {prediction_horizon * 4} ({(i + 1) / (prediction_horizon * 4) * 100:.2f}%)")

        
    #convert predictions and indices to numpy arrays
    predictions_recursive = np.array(predictions_recursive)
    predictions_indices = np.array(predictions_indices)

    #store predictions for this test example
    all_predictions_recursive.append(predictions_recursive)
    all_predictions_indices.append(predictions_indices)
    all_predictions_q95.append(predictions_q95)
    all_predictions_q05.append(predictions_q05)
    
    
    #for the every testset code
    df_result = pd.DataFrame({
        "q05": predictions_q05,
        "q50": predictions_recursive,
        "q95": predictions_q95,
    }, index = pd.Index(predictions_indices.reshape(-1), name = "timestamp"))
    
    all_predictions.append(df_result)

    amount_of_times += 1
    
    if(amount_of_times >= 1000):
        break
    
#convert all predictions to arrays for easier handling
all_predictions_recursive = np.array(all_predictions_recursive)
all_predictions_indices = np.array(all_predictions_indices)
all_predictions_q95 = np.array(all_predictions_q95)
all_predictions_q05 = np.array(all_predictions_q05)

print("Prediction completed for all test samples.")


In [ ]:
all_predictions

In [ ]:
test = all_predictions.copy()

for i, df in enumerate(test):
    timezone = df_data.index.tz  #get timezone from df_data
    df.index = df.index.tz_localize(timezone) if df.index.tz is None else df.index.tz_convert(timezone)
    
    df_merged_all = []
    #merge the two DataFrames on the 'time' index
    df_merged_all = df.merge(df_data[['adjusted_P_max', 'mean_actualPowerTot_W_inverter']], left_index=True, right_index=True, how='left')
    #check for missing values (NaN) in adjusted_P_max
    if df_merged_all['adjusted_P_max'].isna().any():
        print("Warning: Some values are missing in the normalization profile.")

    df_merged_all = df_merged_all.dropna(subset=['adjusted_P_max'])
    
    #denormalize the predicted values columns by multiplying by the 'adjusted_P_max' column
    df_merged_all['denormalized_q50'] = df_merged_all['q50'] * df_merged_all['adjusted_P_max']
    df_merged_all['denormalized_q05'] = df_merged_all['q05'] * df_merged_all['adjusted_P_max']
    df_merged_all['denormalized_q95'] = df_merged_all['q95'] * df_merged_all['adjusted_P_max']

    test[i] = df_merged_all
    
test

In [ ]:
nan_rows = df_merged_all[df_merged_all['adjusted_P_max'].isna()]
print(nan_rows)

In [ ]:
all_results = []

for df in test:
    result = evaluate_benchmark2(df['mean_actualPowerTot_W_inverter'], df['denormalized_q50'])
    
    within_interval = ((df['mean_actualPowerTot_W_inverter'] >= df['denormalized_q05']) &
                   (df['mean_actualPowerTot_W_inverter'] <= df['denormalized_q95']))
    CP = within_interval.mean()

    interval_width = df['denormalized_q95'] - df['denormalized_q05']
    MIW = interval_width.mean()
    relative_interval_width = interval_width / df['mean_actualPowerTot_W_inverter']
    rMIW = relative_interval_width.mean()
    
    if np.isinf(rMIW):
        rMIW = np.nan
    
    result["CP"] = CP
    result["MIW"] = MIW
    result["rMIW"] = rMIW
    
    all_results.append(result)
        

#remove results with 'nan' values for any of the metrics
filtered_results = [r for r in all_results if not any(pd.isna(v) for v in r.values())]

#compute average metrics across all DataFrames
average_results = {
    "Benchmark": "Average Across All",
    "MAPE": round(np.mean([r["MAPE"] for r in filtered_results]), 2),
    "R²": round(np.mean([r["R²"] for r in filtered_results]), 4),
    "MAE": round(np.mean([r["MAE"] for r in filtered_results]), 4),
    "RMSE": round(np.mean([r["RMSE"] for r in filtered_results]), 4),
    "CP": round(np.mean([r["CP"] for r in filtered_results]), 4),
    "MIW": round(np.mean([r["MIW"] for r in filtered_results]), 4),
    "rMIW": round(np.mean([r["rMIW"] for r in filtered_results]), 4),
    "rel_MAE": round(np.mean([r["rel_MAE"] for r in filtered_results]), 4),
    "rel_RMSE": round(np.mean([r["rel_RMSE"] for r in filtered_results]), 4),
}

#print all individual results and the average
for i, res in enumerate(filtered_results):
    print(f"Results for DataFrame {i+1}: {res}")
print("-" * 40)
print("Final Average Results:", average_results)

In [ ]:
dataframe = test[165]

fig = go.Figure()

fig.add_trace(go.Scatter(x=dataframe.index, y=dataframe['denormalized_q50'], mode='lines', name='predicted'))
fig.add_trace(go.Scatter(x=dataframe.index, y=dataframe['mean_actualPowerTot_W_inverter'], mode='lines', name='actual'))
fig.add_trace(go.Scatter(x=dataframe.index, y=dataframe['denormalized_q05'], mode='lines', name='q05',
    line=dict(color='rgba(0,0,255,0)'),
    showlegend=False
))

# Upper quantile (q95) - fill between q05 and q95
fig.add_trace(go.Scatter(x=dataframe.index, y=dataframe['denormalized_q95'],mode='lines',name='90% Interval',
    line=dict(color='rgba(0,0,255,0)'),
    fill='tonexty',  # fill between this trace and the previous (q05)
    fillcolor='rgba(0,0,255,0.2)'
))

# Update layout for better visualization
fig.update_layout(
    title='denormalised',
    xaxis_title='Time',
    yaxis_title='Mean Actual Power (W)',
    xaxis_rangeslider_visible=True
)

fig.show()

In [ ]:
#test other metrics
within_interval = ((dataframe['mean_actualPowerTot_W_inverter'] >= dataframe['denormalized_q05']) &
                   (dataframe['mean_actualPowerTot_W_inverter'] <= dataframe['denormalized_q95']))
CP = within_interval.mean()

interval_width = dataframe['denormalized_q95'] - dataframe['denormalized_q05']
MIW = interval_width.mean()

print("CP: ", CP, " MIW: ", MIW)

In [ ]:
#filter rows where actual power is greater than 0
filtered_df = dataframe[dataframe['mean_actualPowerTot_W_inverter'] > 0]

#calculate CP and MIW on filtered DataFrame
within_interval = ((filtered_df['mean_actualPowerTot_W_inverter'] >= filtered_df['denormalized_q05']) &
                   (filtered_df['mean_actualPowerTot_W_inverter'] <= filtered_df['denormalized_q95']))
CP = within_interval.mean()

interval_width = filtered_df['denormalized_q95'] - filtered_df['denormalized_q05']
MIW = interval_width.mean()
relative_interval_width = interval_width / filtered_df['mean_actualPowerTot_W_inverter']
rMIW = relative_interval_width.mean()


print("filtered CP: ", CP, "filtered MIW: ", MIW, "filtered rMIW: ", rMIW)

In [ ]:
#initialize list of DataFrames for each time step (96 DataFrames for 96 time steps)
time_step_dfs = {i: pd.DataFrame(columns=['actual', 'predicted']) for i in range(output_steps*96)}

#loop over each DataFrame in the test dataset and populate the 96 DataFrames
for df in test:
    #get the 'actual' and 'predicted' values from the DataFrame
    actual_values = df['mean_actualPowerTot_W_inverter'].values
    predicted_values = df['denormalized_q50'].values

    #populate the DataFrames corresponding to each time step
    for i in range(output_steps*96):  #we have 96 time steps (from 0 to 95)
        #create a temporary dataFrame to hold the current actual and predicted values
        temp_df = pd.DataFrame({'actual': [actual_values[i] if i < len(actual_values) else np.nan],
                                'predicted': [predicted_values[i] if i < len(predicted_values) else np.nan]})
        
        time_step_dfs[i] = pd.concat([time_step_dfs[i], temp_df], ignore_index=True)

#evaluate each DataFrame individually using the evaluate_benchmark2 function
all_results = []

for i in range(output_steps*96):
    #get the actual and predicted values for the current time step DataFrame
    step_df = time_step_dfs[i]

    #drop any rows with NaN values
    step_df = step_df.dropna()

    result = evaluate_benchmark2(step_df['actual'], step_df['predicted'])
    result["Time Step"] = i
    
    all_results.append(result)

#compute the average results across all time steps
average_results = {
    "MAPE": round(np.mean([r["MAPE"] for r in all_results]), 2),
    "R²": round(np.mean([r["R²"] for r in all_results]), 4),
    "MAE": round(np.mean([r["MAE"] for r in all_results]), 4),
    "RMSE": round(np.mean([r["RMSE"] for r in all_results]), 4),
}

for res in all_results:
    print(f"Results for Time Step {res['Time Step']}: {res}")

print("-" * 40)
print("Final Average Results:", average_results)

In [ ]:
#extract the metrics for plotting
time_steps = [r["Time Step"] for r in all_results]
mape_values = [r["MAPE"] for r in all_results]
mae_values = [r["MAE"] for r in all_results]
rmse_values = [r["RMSE"] for r in all_results]
r2_values = [r["R²"] for r in all_results]

#create subplots
fig = go.Figure()

#plot MAPE
#fig.add_trace(go.Scatter(x=time_steps, y=mape_values, mode='lines', name='MAPE', line=dict(color='blue')))

#plot MAE
fig.add_trace(go.Scatter(x=time_steps, y=mae_values, mode='lines', name='MAE', line=dict(color='green')))

#plot RMSE
#fig.add_trace(go.Scatter(x=time_steps, y=rmse_values, mode='lines', name='RMSE', line=dict(color='red')))

#plot R²
#fig.add_trace(go.Scatter(x=time_steps, y=r2_values, mode='lines', name='R²', line=dict(color='orange')))

fig.update_layout(
    title="Evaluation Metrics Across Time Steps for MAE",
    xaxis_title="Time Step",
    yaxis_title="MAE Value (W)",
    legend_title="Metrics"
)

fig.show()

file_path = r"C:\Users\samr0\OneDrive - KU Leuven\Documents\!School\master\Thesis\data\timestep_results_quantile_recursive_test4.csv"
df_results = pd.DataFrame(all_results)
df_results.to_csv(file_path, index=False)